In [1]:
%reload_ext autoreload
%autoreload 2

import os

import pandas as pd
from dotenv import load_dotenv

from src import ghost, html
from src.evaluation import eval_recipes
from src.models import Recipe

### Setup enviroment

In [2]:
load_dotenv()
LITELLM_API_KEY = os.getenv("LITELLM_API_KEY")
MODEL = "gpt-4o-mini"
LITELLM_URL = os.getenv("LITELLM_URL")

In [3]:
prompt = "Extract the recipe from the following html - {content}"

### HTML modification here

### Eval

In [6]:
true_html = html.parse_from_file("src/data/dummy/recipe_1.html")
modified_html = html.parse_from_file("src/data/dummy/recipe_1.html")


score = ghost.compare(
    # true html
    true_html=true_html,
    # modified html
    modified_html=modified_html,
    # prompt for llm
    prompt=prompt,
    # response object that should be extracted from html
    response_object=Recipe,
    # eval function to evaluate the response object
    eval_func=eval_recipes,
    # api key for litellm
    api_key=LITELLM_API_KEY,
    # litellm base url
    base_url=LITELLM_URL,
)
score

{'title_match': True,
 'title_distance': 0,
 'ingredients_match': True,
 'ingredients_distance': 0,
 'instructions_match': True,
 'instructions_distance': 0}

In [5]:
pd.DataFrame([score])

,title_match,title_distance,ingredients_match,ingredients_distance,instructions_match,instructions_distance
0,True,0,True,0,True,0
